In [1]:
import pandas as pd
import os
import pickle
import numpy as np
import glob
from datetime import date
import datetime
import torch
import time
import sys
import tqdm
from scipy.special import softmax
from datasets import Dataset
# Preprocessing
import transformers
import datasets
import scipy

In [2]:
data_dir = '/Users/yujia/research/stance/data/all_posts.pickle'
posts = pd.read_pickle(data_dir)
print(len(posts))
posts.head()

1046958


,created_utc,id,author,title,selftext,subreddit,link,score,upvote_ratio,author_flair_text,num_comments,removed_by_category,url,divisiveness
0,1588242127,gat3o9,TransGirl757,"If we get approved by 100k people in 1 month, ...",NaN,progressive,/r/progressive/comments/gat3o9/if_we_get_appro...,1,1.0,NaN,0,moderator,https://petitions.whitehouse.gov/petition/give...,1
1,1588222576,gapbez,shallah,Elizabeth Warren and Alexandria Ocasio-Cortez ...,NaN,progressive,/r/progressive/comments/gapbez/elizabeth_warre...,1,1.0,NaN,0,moderator,https://theweek.com/speedreads/911377/elizabet...,1
2,1588216466,ganw7m,shallah,"Gov Whitmer Announces ""Futures for Frontliners...",NaN,progressive,/r/progressive/comments/ganw7m/gov_whitmer_ann...,1,1.0,NaN,5,NaN,"https://www.michigan.gov/whitmer/0,9309,7-387-...",1
3,1588200464,gajmqr,thinkB4WeSpeak,Could pandemic rekindle labor movement?,NaN,progressive,/r/progressive/comments/gajmqr/could_pandemic_...,1,1.0,NaN,2,NaN,https://finance-commerce.com/2020/04/could-pan...,1
4,1588197018,gaikq1,davyjoneses,Found on r/therightcantmeme. Should the govern...,NaN,progressive,/r/progressive/comments/gaikq1/found_on_rtheri...,1,1.0,NaN,1,moderator,https://np.reddit.com/r/TheRightCantMeme/comme...,1


In [18]:
file = "20200813_20201025"
fname = '/Users/yujia/research/stance/data/comments/{}_all_comments_with_labels.pickle'.format(file)
comments = pd.read_pickle(fname)
print(len(comments))
comments.head()

7619033


,created_utc,parent_id,link_id,id,author,body,subreddit,link,score,author_flair_text,...,social,you,negate,negemo,sad,posemo,anger,swear,we,i
0,1603597767,t1_g9sut4x,t3_jgovwn,ga03hb8,ShmeowShmeow,Do you dispute that Hunter was getting paid la...,uspolitics,/r/uspolitics/comments/jgovwn/no_evidence_for_...,1,NaN,...,1,1,0,0,0,0,0,0,0,0
1,1603597174,t3_jhfcqw,t3_jhfcqw,ga01x2j,BIGBOSS7007,"I mean, Trump can pay a PMC to try to stage a ...",uspolitics,/r/uspolitics/comments/jhfcqw/what_can_you_do_...,1,NaN,...,2,0,1,1,1,0,0,0,0,0
2,1603596218,t1_g9z6dyy,t3_jhc9bm,g9zzcy9,sulaymanf,Billboards are not FCC jurisdiction.,uspolitics,/r/uspolitics/comments/jhc9bm/lawyers_for_ivan...,1,NaN,...,0,0,1,0,0,0,0,0,0,0
3,1603594598,t3_jhm13a,t3_jhm13a,g9zv3vb,votebymailbot2020,# Texas Voter Information\n\n## Voter Registra...,uspolitics,/r/uspolitics/comments/jhm13a/texas_governor_r...,1,NaN,...,0,0,1,0,0,0,0,0,0,0
4,1603591908,t1_g9wqazp,t3_jh39dd,g9znyf2,kkdawg79,Ruler to the knuckles at the Kremlin,uspolitics,/r/uspolitics/comments/jh39dd/mcconnell_says_n...,2,NaN,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# 1584743622
comments = comments[comments.created_utc > 1584743622] 

In [5]:
comments['comment_len'] = comments['body'].apply(lambda x: len(x.split()))
comments.to_pickle(fname)
comments.head()

,created_utc,parent_id,link_id,id,author,body,subreddit,link,score,author_flair_text,vulgarity,personal_attacks,incivility,third_party_attacks,aspersion,stereotype,comment_len
0,1594611009,t3_hq7drs,t3_hq7drs,fxw8pc6,Koalemos78,Interesting take from someone who's education ...,uspolitics,/r/uspolitics/comments/hq7drs/nm_state_senator...,3,NaN,0.201874,0.002277,0.011276,0.004292,0.000131,0.484231,31
1,1594610955,t1_fxl5isk,t3_hovua5,fxw8mi5,Thatsprettydank,Kinda lol they stopped researching out of bigotry,uspolitics,/r/uspolitics/comments/hovua5/well_be_living_w...,1,NaN,0.114368,0.002072,0.139416,0.727760,0.000138,0.276559,8
3,1594608113,t1_fxw2l39,t3_hq16qw,fxw4axz,Peripatetic_deviant,So you’re just guessing information? Even if t...,uspolitics,/r/uspolitics/comments/hq16qw/feds_in_portland...,1,NaN,0.147343,0.025804,0.901238,0.003068,0.000791,0.448298,19
4,1594607019,t1_fxvrvdq,t3_hq16qw,fxw2l39,buffalo_chum,They were out past curfew no? I bet there was ...,uspolitics,/r/uspolitics/comments/hq16qw/feds_in_portland...,1,NaN,0.110046,0.001438,0.014784,0.004238,0.000134,0.332198,17
5,1594606124,t3_hq4pq6,t3_hq4pq6,fxw153j,metaltimmy,Trump always likes these photo ops where he th...,uspolitics,/r/uspolitics/comments/hq4pq6/trump_ripped_as_...,5,NaN,0.378780,0.001770,0.994990,0.976932,0.999549,0.230994,176


In [19]:
survival_data_fname = '/Users/yujia/research/civility/datasets/survival_data_more.pickle'
if os.path.isfile(survival_data_fname):
    survival_data = pd.read_pickle(survival_data_fname)
survival_data = survival_data.reset_index()
survival_data = set(survival_data.sort_values('author').author)
survival_data = pd.DataFrame(survival_data,columns=['author']).sort_values('author').reset_index()
survival_data

,index,author
0,69667,-----iMartijn-----
1,2445,---00G---
2,103028,---77---
3,19488,---Tsing__Tao---
4,92313,---deadman---
...,...,...
119047,76603,zzziltoid
119048,74038,zzzpranger
119049,107984,zzzwrite
119050,55439,zzzz1234zzzz


In [17]:
liwc = np.load('/Users/yujia/research/civility/data/liwc.npy', allow_pickle=True)
# categories = list(liwc.item().keys())
# print(categories)
categories = ['affect', 'they', 'insight', 'social', 'anger', 'swear', 'sad', 'negate', 'i', 'you', 'we', 'posemo', 'negemo']
# trie = build_category_trie(liwc, categories)
# comments['categories'] = comments.tokenized.apply(lambda l : search_trie_for_list(trie, l))
liwc_scores = comments.categories.apply(pd.Series).fillna(0).astype(int)
comments = pd.concat([comments, liwc_scores], axis=1)
comments

,created_utc,parent_id,link_id,id,author,body,subreddit,link,score,author_flair_text,...,negate,they,you,affect,posemo,negemo,anger,sad,we,swear
0,1594611009,t3_hq7drs,t3_hq7drs,fxw8pc6,Koalemos78,Interesting take from someone who's education ...,uspolitics,/r/uspolitics/comments/hq7drs/nm_state_senator...,3,NaN,...,1,0,0,0,0,0,0,0,0,0
1,1594610955,t1_fxl5isk,t3_hovua5,fxw8mi5,Thatsprettydank,Kinda lol they stopped researching out of bigotry,uspolitics,/r/uspolitics/comments/hovua5/well_be_living_w...,1,NaN,...,0,1,0,0,0,0,0,0,0,0
3,1594608113,t1_fxw2l39,t3_hq16qw,fxw4axz,Peripatetic_deviant,So you’re just guessing information? Even if t...,uspolitics,/r/uspolitics/comments/hq16qw/feds_in_portland...,1,NaN,...,0,0,1,0,0,0,0,0,0,0
4,1594607019,t1_fxvrvdq,t3_hq16qw,fxw2l39,buffalo_chum,They were out past curfew no? I bet there was ...,uspolitics,/r/uspolitics/comments/hq16qw/feds_in_portland...,1,NaN,...,1,0,0,0,0,0,0,0,0,0
5,1594606124,t3_hq4pq6,t3_hq4pq6,fxw153j,metaltimmy,Trump always likes these photo ops where he th...,uspolitics,/r/uspolitics/comments/hq4pq6/trump_ripped_as_...,5,NaN,...,1,1,2,18,11,7,3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4497709,1590724825,t1_fs5ydt2,t3_gs7x4m,fs5yduc,AutoModerator,Your [comment](https://www.reddit.com/r/Libert...,Libertarian,/r/Libertarian/comments/gs7x4m/would_you_vote_...,1,What is my purpose? You ban memes...,...,1,1,2,0,0,0,0,0,1,0
4497711,1590724817,t1_fs5tuvq,t3_gshko8,fs5yde6,carnagebestvillain,Ah makes sense. Youre an authoritarian.\n\nLin...,Libertarian,/r/Libertarian/comments/gshko8/why_the_hell_ar...,1,NaN,...,0,0,2,1,0,1,1,0,0,0
4497712,1590724816,t3_gshb29,t3_gshb29,fs5ydbk,Effotless,The goal of the government is to protect our r...,Libertarian,/r/Libertarian/comments/gshb29/im_a_libertaria...,1,Anarcho-Objectivist,...,1,2,0,0,0,0,0,0,2,0
4497713,1590724814,t1_fs5vxr0,t3_gsieg3,fs5yd7o,Julian_Caesar,...well said.,Libertarian,/r/Libertarian/comments/gsieg3/the_minneapolis...,1,NaN,...,0,0,0,1,1,0,0,0,0,0


In [20]:
from datetime import datetime, timedelta
from scipy import stats
import time

feb_2020_timestamp = 1580533200
feb_2021_timestamp = 1612155600

count = 0

# days = [29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 30]
timestamps = [datetime.utcfromtimestamp(feb_2020_timestamp)]
for i in range(1, 53):
  timestamps.append( timestamps[i-1] + timedelta(days=7))

timestamps = [i.timestamp() for i in timestamps]
print(timestamps)

def compute_binned_statistics(exp, fields, row, prefix="sent",use_liwc=False, liwc=None):
  global count
  values = []
  for f in fields:
    values.append(exp[f])
  if use_liwc:
    fields.extend(liwc.columns)
    for i in liwc.columns:
      values.append(liwc[i])
  res = stats.binned_statistic(exp['created_utc'], values, 'mean', bins=timestamps)[0]
  for index, r in enumerate(res):
    row[prefix+"_"+fields[index]+"_hist"] = np.nan_to_num(r)
  
categories = ['vulgarity', 'third_party_attacks', 'stereotype', 'personal_attacks', 'incivility', 'aspersion']

def find_comment_activities(posts_df, comments_df, row):
  global count, program_starts
  if count % 100 == 0:
    now = time.time()
    print("It has been {} seconds since the loop started to get to {}".format(now - program_starts, count))
    program_starts = time.time()
  count += 1
  # exposure
  temp_posts = posts_df[posts_df.author == row.author]
  row['post_week_hist'] = np.histogram(temp_posts['created_utc'], bins=timestamps)[0]
  # if len(temp_posts) > 0:
  #   res = stats.binned_statistic(temp_posts['created_utc'], [temp_posts['is_left'], temp_posts['is_right']], 'sum', bins=timestamps)[0]
  #   row['left_post_hist'] = res[0]
  #   row['right_post_hist'] = res[1]

  temp_df = comments_df[comments_df.link_id.str[3:].isin(temp_posts.id)]
  row['post_comment_num_hist'] = np.histogram(temp_df['created_utc'], bins=timestamps)[0]
  if len(temp_df) > 0:
    compute_binned_statistics(temp_df,  categories, row, 'post')

  # comments
  temp_df = comments_df[comments_df.author == row.author]
  row['comment_week_hist'] = np.histogram(temp_df['created_utc'], bins=timestamps)[0]
  if len(temp_df) > 0:
  #   liwc = temp_df.categories.apply(pd.Series).fillna(0).astype(int)
    compute_binned_statistics(temp_df,  categories, row, 'sent')

    # immediate response
    response_df = comments_df[comments_df.parent_id.str[3:].isin(temp_df.id)]
    if len(response_df) > 0:
      row['comment_response_num_hist'] = np.histogram(temp_df['created_utc'], bins=timestamps)[0]
      compute_binned_statistics(response_df, categories, row, 'comment')
  return row

[1580551200.0, 1581156000.0, 1581760800.0, 1582365600.0, 1582970400.0, 1583575200.0, 1584176400.0, 1584781200.0, 1585386000.0, 1585990800.0, 1586595600.0, 1587200400.0, 1587805200.0, 1588410000.0, 1589014800.0, 1589619600.0, 1590224400.0, 1590829200.0, 1591434000.0, 1592038800.0, 1592643600.0, 1593248400.0, 1593853200.0, 1594458000.0, 1595062800.0, 1595667600.0, 1596272400.0, 1596877200.0, 1597482000.0, 1598086800.0, 1598691600.0, 1599296400.0, 1599901200.0, 1600506000.0, 1601110800.0, 1601715600.0, 1602320400.0, 1602925200.0, 1603530000.0, 1604134800.0, 1604743200.0, 1605348000.0, 1605952800.0, 1606557600.0, 1607162400.0, 1607767200.0, 1608372000.0, 1608976800.0, 1609581600.0, 1610186400.0, 1610791200.0, 1611396000.0, 1612000800.0]


In [23]:
survival_data_dup = survival_data.drop('index', axis=1)
survival_data_dup = survival_data_dup.reset_index()

file_dir = '/Users/yujia/research/civility/data/survival_temp/'+file
print(file_dir)
if not os.path.isdir(file_dir):
  os.mkdir(file_dir)
i = 0
step_size = 10
program_starts = time.time()
count = 40570
for i in range(count, len(survival_data_dup), step_size):
  # print("starting with " + str(i))
#   survival_data_dup['author'] = survival_data_dup['author']
  survival_data_cur = survival_data_dup.loc[i:i+step_size-1]
  posts_cur = posts[posts.author.isin(survival_data_cur.author)]
  comments_cur = comments[(comments.author.isin(survival_data_cur.author))]
  comments_cur = comments[(comments.author.isin(survival_data_cur.author) | comments.link_id.isin(posts_cur.id) | comments.link_id.isin(comments_cur.link_id))]
  survival_data_cur = survival_data_cur.apply(lambda row: find_comment_activities(posts_cur, comments_cur, row), axis=1)
  survival_data_cur.to_pickle(file_dir +'/survival_data_{}.pickle'.format(i))
  # print("done")

/Users/yujia/research/civility/data/survival_temp/20200813_20201025
It has been 12.014631986618042 seconds since the loop started to get to 40600
It has been 36.269737005233765 seconds since the loop started to get to 40700
It has been 27.77193307876587 seconds since the loop started to get to 40800
It has been 25.05873680114746 seconds since the loop started to get to 40900
It has been 26.745211124420166 seconds since the loop started to get to 41000
It has been 24.910075902938843 seconds since the loop started to get to 41100
It has been 25.138052940368652 seconds since the loop started to get to 41200
It has been 32.699477195739746 seconds since the loop started to get to 41300
It has been 30.38550090789795 seconds since the loop started to get to 41400
It has been 26.99254608154297 seconds since the loop started to get to 41500
It has been 31.76866912841797 seconds since the loop started to get to 41600
It has been 28.15024209022522 seconds since the loop started to get to 41700
It

: 